In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/Colab Notebooks/RBM - CF/Recommender-System

/content/gdrive/MyDrive/Colab Notebooks/RBM - CF/Recommender-System


In [20]:
from collaborative_filtering import CF 
from demographic_filtering import DF
from perceptron import Perceptron as pct
from get_data import (
    get_users_data,
    get_rating_base_data,
    get_rating_test_data,
)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch

# Import RBM model script
from rbm import RBM


In [4]:
# CF 
RATE_TRAIN = get_rating_base_data().values # convert to matrix
RATE_TEST = get_rating_test_data().values # convert to matrix

RATE_TRAIN[:, :2] -= 1 # start from 0
RATE_TEST[:, :2] -= 1

CF = CF(RATE_TRAIN, k=25)
CF.fit()

print('Similar Matrix Works')
print(CF.S)
print('Number of rows: ', CF.S.shape[0])
print('Number of columns: ', CF.S.shape[1])

ids = np.where(RATE_TEST[:, 0] == 0)[0].astype('int32')
real_items_1 = RATE_TEST[(np.where((RATE_TEST[:, 0] == 0) & (RATE_TEST[:, 2] >= 3)))]
predicted_items = []

for row in RATE_TEST[ids, :]:
    predicted_rating = CF.pred(0, row[1])
    if predicted_rating >= 3:
        predicted_items.append(row[1])

print('Items which user 1 actually like: ', real_items_1[:, 1])
print('Items in prediction which user 1 might like ', predicted_items)


n_test = RATE_TEST.shape[0]
correct_items_count = 0
real_items_user_like_count = len(np.where(RATE_TEST[:, 2] >= 3)[0].astype(np.int32))

user_id = 0
while user_id < CF.n_users:
    ids = np.where(RATE_TEST[:, 0] == user_id)[0].astype('int32')
    real_items = RATE_TEST[(np.where((RATE_TEST[:, 0] == user_id) & (RATE_TEST[:, 2] >= 3)))]
    for row in RATE_TEST[ids, :]:
        predicted_rating = CF.pred(user_id, row[1])
        if predicted_rating >= 3 and row[1] in real_items:
            correct_items_count += 1
    user_id += 1
accuracy = correct_items_count/real_items_user_like_count
print('The accuracy of Collaborative Filtering: {0}/{1} = {2}'.format(correct_items_count, real_items_user_like_count, accuracy))

Similar Matrix Works
[[ 1.   -0.08  0.01 ... -0.08  0.03 -0.06]
 [-0.08  1.    0.05 ... -0.11  0.    0.22]
 [ 0.01  0.05  1.   ... -0.01 -0.04  0.05]
 ...
 [-0.08 -0.11 -0.01 ...  1.    0.   -0.18]
 [ 0.03  0.   -0.04 ...  0.    1.    0.  ]
 [-0.06  0.22  0.05 ... -0.18  0.    1.  ]]
Number of rows:  943
Number of columns:  943
Items which user 1 actually like:  [  0   1   8  21  25  29  31  37  39  67  74  88  92  98 118 132 134 135
 140 143 145 146 157 165 172 175 178 180 186 193 196 204 210 219 233 238
 245 248 255 256 267 268 269]
Items in prediction which user 1 might like  [0, 1, 7, 8, 20, 21, 25, 29, 31, 33, 36, 37, 39, 62, 67, 74, 88, 92, 98, 104, 118, 130, 132, 134, 135, 137, 140, 143, 145, 146, 157, 165, 172, 175, 178, 180, 186, 193, 196, 204, 210, 219, 233, 236, 238, 245, 248, 255, 256, 267, 268, 269, 270]
The accuracy of Collaborative Filtering: 14963/16558 = 0.903671941055683


In [57]:
from sklearn.metrics import recall_score, precision_score
x = list(set(get_rating_base_data().iloc[:,1]))
x = pd.DataFrame(columns=x)
# lấy dữ liệu của những iteam thực sự được người dùng thích
zipper = np.ones(len(real_items_1[:, 1]))
dict_ = dict(zip(real_items_1[:, 1], zipper))
y_real = x.append(dict_,ignore_index=True).fillna(0).values

# lấy dữ liệu của những item predict
zipper = np.ones(len(predicted_items))
dict_ = dict(zip(predicted_items, zipper))
y_pred = x.append(dict_,ignore_index=True).fillna(0).values

recall = recall_score(y_real, y_pred, average='macro')
precision = precision_score(y_real, y_pred, average='macro')

print('Chỉ số recall: {}\nChỉ số precision: {}'.format(recall, precision))

Chỉ số recall: 0.02607640994542147
 Chỉ số precision: 0.02607640994542147


In [59]:
# DF
RATE_TRAIN = get_rating_base_data().values # convert to matrix
RATE_TEST = get_rating_test_data().values # convert to matrix

RATE_TRAIN[:, :2] -= 1 # start from 0
RATE_TEST[:, :2] -= 1
USERS = get_users_data()
DF = DF(USERS, RATE_TRAIN, 25)
DF.fit()

print("Ma trận tương đồng thông tin")
print(DF.similarities)
print("Số hàng của ma trận:", DF.similarities.shape[0])
print("Số cột của ma trận: ", DF.similarities.shape[1])

ids = np.where(RATE_TEST[:, 0] == 0)[0].astype("int32")
real_items_1 = RATE_TEST[(np.where((RATE_TEST[:, 0] == 0) & (RATE_TEST[:, 2] >= 3)))]
correct_predicted_items = []

for row in RATE_TEST[ids, :]:
    predicted_rating = DF.pred(0, row[1])
    if predicted_rating >= 3 and row[1] in real_items_1:
        correct_predicted_items.append(row[1])


print("Những items user 1 thật sự thích       : ", real_items_1[:, 1])
print("Những items user 1 được dự đoán thích  : ", correct_predicted_items)

n_test = RATE_TEST.shape[0]
correct_items_count = 0
real_items_user_like_count = len(np.where(RATE_TEST[:, 2] >= 3)[0].astype(np.int32))

user_id = 0
while user_id < DF.n_users:
    ids = np.where(RATE_TEST[:, 0] == user_id)[0].astype("int32")
    real_items = RATE_TEST[(np.where((RATE_TEST[:, 0] == user_id) & (RATE_TEST[:, 2] >= 3)))]
    for row in RATE_TEST[ids, :]:
        predicted_rating = DF.pred(user_id, row[1])
        if predicted_rating >= 3 and row[1] in real_items:
            correct_items_count = correct_items_count + 1
    user_id = user_id + 1

print("Độ chính xác của Demographic Filtering :", correct_items_count / real_items_user_like_count)

Ma trận tương đồng thông tin
[[1.         0.         0.66666667 ... 0.66666667 0.         0.66666667]
 [0.         1.         0.         ... 0.         0.33333333 0.        ]
 [0.66666667 0.         1.         ... 0.66666667 0.         0.66666667]
 ...
 [0.66666667 0.         0.66666667 ... 1.         0.         1.        ]
 [0.         0.33333333 0.         ... 0.         1.         0.        ]
 [0.66666667 0.         0.66666667 ... 1.         0.         1.        ]]
Số hàng của ma trận: 943
Số cột của ma trận:  943
Những items user 1 thật sự thích       :  [  0   1   8  21  25  29  31  37  39  67  74  88  92  98 118 132 134 135
 140 143 145 146 157 165 172 175 178 180 186 193 196 204 210 219 233 238
 245 248 255 256 267 268 269]
Những items user 1 được dự đoán thích  :  [0, 1, 8, 21, 25, 29, 31, 37, 67, 74, 88, 92, 98, 118, 132, 134, 135, 140, 143, 146, 165, 172, 175, 178, 180, 186, 193, 196, 204, 210, 219, 233, 238, 245, 248, 255, 256, 267, 268, 269]
Độ chính xác của Demographic Fil

In [ ]:
# Perceptron
ids = np.where(RATE_TEST[:, 0] == 0)[0].astype("int32")

MATRIX_DF = []
MATRIX_CF = []

for row in RATE_TEST[ids, :]:
    p_cf = CF.pred(0, row[1])
    p_df = DF.pred(u=0, i=row[2])
    MATRIX_CF.append([0, row[1], p_cf])
    MATRIX_DF.append([0, row[1], p_df])
MATRIX_CF = np.asarray(MATRIX_CF)
MATRIX_DF = np.asarray(MATRIX_DF)

CF_predicted = np.asanyarray(MATRIX_CF[:, 2])
DF_predicted = MATRIX_DF[:, 2]
true_rating = RATE_TEST[ids, 2]

dataset = np.c_[CF_predicted, DF_predicted, true_rating]

# print("Ma trận dự đoán đánh giá CF, DF, True Rating")
# print(dataset)

PLA = Perceptron(dataset, 0.003, len(ids))
PLA.fit()
predicted_ratings_pla = PLA.predict()

print("Dự đoán đánh giá sau khi được điều chỉnh")
print(np.round(predicted_ratings_pla, 3))


In [65]:
ratings = get_rating_base_data().iloc[:, :3]
ratings_pivoted = pd.pivot_table(ratings, values='rating', index=['user_id'],
                    columns=['item_id'], aggfunc=np.mean).fillna(0)

item_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,4.0,3.0,3.0,5.0,4.0,0.0,0.0,3.0,2.0,5.0,5.0,5.0,5.0,5.0,3.0,4.0,5.0,4.0,0.0,0.0,4.0,3.0,4.0,0.0,2.0,4.0,1.0,0.0,3.0,0.0,4.0,0.0,1.0,2.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:


ratings_pivoted = ratings_pivoted.apply(lambda x: 1 if x >= 3 else (0 if x > 0 and x < 3 else -1))
ratings_pivoted

ModuleNotFoundError: ignored

In [67]:
!git pull

remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 9 (delta 5), reused 9 (delta 5), pack-reused 0
Unpacking objects: 100% (9/9), done.
From https://github.com/phusdt/Recommender-System
   d47efa1..fad3396  master     -> origin/master
Updating d47efa1..fad3396
error: Your local changes to the following files would be overwritten by merge:
	RS.ipynb
	__pycache__/demographic_filtering.cpython-37.pyc
	demographic_filtering.py
Please commit your changes or stash them before you merge.
Aborting


In [69]:
!git add .



*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@ef5477c80efe.(none)')


In [77]:
!git config --global user.email 'phuduytran1007@gmail.com'

In [80]:
 !git config --global user.name "phusdt"

In [81]:
!git commit -m 'update rs'

[master 29d7e08] update rs
 6 files changed, 8 insertions(+), 258 deletions(-)
 rewrite RS.ipynb (100%)
 create mode 100644 Untitled0.ipynb
 mode change 100755 => 100644 ml-100k/allbut.pl
 mode change 100755 => 100644 ml-100k/mku.sh


In [86]:
!git pull 'rbm.py'

fatal: 'rbm.py' does not appear to be a git repository
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.
